## TensorFlow

In [1]:
import tensorflow as tf

In [2]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Data preprocessing
x_train = tf.keras.applications.resnet50.preprocess_input(x_train.astype('float32'))
x_test = tf.keras.applications.resnet50.preprocess_input(x_test.astype('float32'))

In [3]:
# Initialize the model, excluding the last fully connected layer
resnet50 = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Freeze all the layers
for layer in resnet50.layers:
    layer.trainable = False

# Build model
inputs = tf.keras.layers.Input(shape=(32,32,3))
x = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
x = resnet50(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)

# Compile the model
model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2  (None, 224, 224, 3)       0         
 D)                                                              
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              209817

In [5]:
# Train
model.fit(x_train, y_train, batch_size=64, epochs=3)

Epoch 1/3
782/782 [==============================] - 152s 183ms/step - loss: 0.8371 - accuracy: 0.7096
Epoch 2/3
782/782 [==============================] - 142s 182ms/step - loss: 0.5959 - accuracy: 0.7915
Epoch 3/3
782/782 [==============================] - 142s 182ms/step - loss: 0.5284 - accuracy: 0.8168


In [6]:
# Evaluation
model.evaluate(x_test, y_test)

313/313 [==============================] - 32s 95ms/step - loss: 0.7467 - accuracy: 0.7520


[0.7466651201248169, 0.7519999742507935]

## PyTorch

In [7]:
from torchvision.models import resnet50, ResNet50_Weights
import torchvision.transforms as transforms
import torchvision
from torch import nn
import torch

In [8]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Load Data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,
                                             download=True,
                                             transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=False,
                                             transform=transform)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64,
                                           shuffle=True)

Files already downloaded and verified


In [10]:
# Load pretrained ResNet50 model
model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)

# Freeze all layers in ResNet50
for param in model.parameters():
    param.requires_grad = False

# Fully connected layer
model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
).to(device)

# Optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [11]:
# Train
train_loss = 0.0
train_acc = 0.0
num_epochs = 3

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()

    # Print metrics
    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    print(f'Training Loss: {train_loss:.4f}  |  Training Accuracy: {train_acc:.4f}')

Epoch 1/3
Training Loss: 0.0270  |  Training Accuracy: 0.3990
Epoch 2/3
Training Loss: 0.0245  |  Training Accuracy: 0.4559
Epoch 3/3
Training Loss: 0.0235  |  Training Accuracy: 0.4769


In [12]:
# Evaluation
val_loss = 0.0
val_acc = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        # Forward pass
        val_loss += criterion(outputs, labels)
        val_acc += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(test_loader.dataset)
    val_acc /= len(test_loader.dataset)

    print(f' Validation Loss: {val_loss:.4f}  |  Validation Accuracy: {val_acc:.4f}')

 Validation Loss: 0.0246  |  Validation Accuracy: 0.4560
